In [1]:
from langgraph.graph import StateGraph , START , END
from langchain_ollama import ChatOllama
from typing import Annotated , TypedDict
from langgraph.checkpoint.memory import InMemorySaver
from dotenv import load_dotenv

In [2]:
load_dotenv()

llm = ChatOllama(model='gemma:2b')

In [3]:
class JokeState(TypedDict):
    
    topic : str
    joke : str
    explanation : str

In [4]:
def generate_joke(state : JokeState) -> JokeState:
    
    prompt = f"Generate a joke about {state['topic']}"
    
    response = llm.invoke(prompt).content
    
    return {'joke' : response}

In [5]:
def generate_explanation(state : JokeState) -> JokeState:
    
    prompt = f"Explain the joke '{state['joke']}'"

    response = llm.invoke(prompt).content

    return {'explanation' : response}

In [7]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke' , generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START , 'generate_joke')
graph.add_edge('generate_joke' , 'generate_explanation')
graph.add_edge('generate_explanation' , END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [13]:
config1 = {'configurable': {'thread_id':'1'}}
workflow.invoke({'topic':'Education'}, config=config1)

{'topic': 'Education',
 'joke': "What do you call an education that's too easy?\n\nA GED!",
 'explanation': 'The joke is about the irony of the GED test. It is a test that is designed to measure a person\'s education, yet the answer to the question "What do you call an education that\'s too easy?" is simply "A GED!".'}

In [12]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'Education', 'joke': "What do you call an education that's too easy?\n\nA textbook!", 'explanation': 'The joke is a play on words that uses the literal meaning of the words "textbook" and "easy". The textbook is a book that is used for learning, and an easy problem is a problem that is easy to solve. The joke is that the textbook is literally a book that is too easy, and therefore cannot be used to learn anything.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f078354-3c9c-62c5-8006-a578aacfd64d'}}, metadata={'source': 'loop', 'step': 6, 'parents': {}}, created_at='2025-08-13T11:04:21.438125+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f078353-a70b-62aa-8005-8fd6cabb4d7a'}}, tasks=(), interrupts=())

In [15]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'Education', 'joke': "What do you call an education that's too easy?\n\nA GED!", 'explanation': 'The joke is about the irony of the GED test. It is a test that is designed to measure a person\'s education, yet the answer to the question "What do you call an education that\'s too easy?" is simply "A GED!".'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f07835a-00d4-6b6f-800a-9936601ee38e'}}, metadata={'source': 'loop', 'step': 10, 'parents': {}}, created_at='2025-08-13T11:06:56.231102+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f078359-4c27-6bdb-8009-287144d12222'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'Education', 'joke': "What do you call an education that's too easy?\n\nA GED!", 'explanation': 'The joke is a play on words that uses the literal meaning of the words "textbook" and "easy". The textbook is a book that is used for lear

In [16]:
config2 = {'configurable': {'thread_id':'2'}}
workflow.invoke({'topic':'School'}, config=config2)

{'topic': 'School',
 'joke': "What do you call a teacher who's too strict?\n\nA stickler for classroom rules!",
 'explanation': "The joke suggests that a strict teacher is inflexible and overly demanding. It implies that the teacher's rules and expectations are so strict that they restrict the students' freedom of expression and creativity."}

In [17]:
workflow.get_state(config2)

StateSnapshot(values={'topic': 'School', 'joke': "What do you call a teacher who's too strict?\n\nA stickler for classroom rules!", 'explanation': "The joke suggests that a strict teacher is inflexible and overly demanding. It implies that the teacher's rules and expectations are so strict that they restrict the students' freedom of expression and creativity."}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f078376-5c63-6916-8002-45ce7df91e49'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-13T11:19:37.450933+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f078375-cced-6d30-8001-fc95b112c184'}}, tasks=(), interrupts=())

In [18]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'School', 'joke': "What do you call a teacher who's too strict?\n\nA stickler for classroom rules!", 'explanation': "The joke suggests that a strict teacher is inflexible and overly demanding. It implies that the teacher's rules and expectations are so strict that they restrict the students' freedom of expression and creativity."}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f078376-5c63-6916-8002-45ce7df91e49'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-13T11:19:37.450933+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f078375-cced-6d30-8001-fc95b112c184'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'School', 'joke': "What do you call a teacher who's too strict?\n\nA stickler for classroom rules!"}, next=('generate_explanation',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id